In [ ]:
# @title  {"display-mode":"form"}
#@markdown <center><h2><b>Vimeo Video Downloader</b></h2>Download Vimeo videos easy v0.2</center><br>
#@markdown <center>Authored by: limontec (https://github.com/limontec)<br />Modified by: Bartek20 (https://github.com/Bartek20)</center>

import os
import sys
import base64
import requests
import subprocess
from google.colab import files
from tqdm import tqdm

class VimeoDownloader:
  def __init__(self, url, filename):
    if 'master.json' in url:
      url = url[:url.find('?')] + '?query_string_ranges=1'
      url = url.replace('master.json', 'master.mpd')
    self.url = url
    if not filename.endswith('.mp4'):
      filename += '.mp4'
    self.filename = filename
  def save(self):
    try:
      files.download(self.filename)
      print("If download does not start, download the file manually from the directory in the left corner.")
    except FileNotFoundError:
      print("File not found!")
    except:
      print("Something else went wrong, try download manually")
  def fetch(self, what, to, base):
    with open(to, 'wb') as file:
      init_segment = base64.b64decode(what['init_segment'])
      file.write(init_segment)
      for segment in tqdm(what['segments']):
        segment_url = base + segment['url']
        resp = requests.get(segment_url, stream=True)
        if resp.status_code != 200:
          print('Segment not found:', segment_url)
          break
        for chunk in resp:
            file.write(chunk)
  def download(self):
    if 'master.mpd' in self.url:
      subprocess.run(['youtube-dl', self.url, '-o', self.filename])
      return
    base_url = self.url[:self.url.rfind('/', 0, -26) + 1]
    content = requests.get(self.url).json()
    vid_heights = [(i, d['height']) for (i, d) in enumerate(content['video'])]
    vid_idx, _ = max(vid_heights, key=lambda _h: _h[1])
    audio_quality = [(i, d['bitrate']) for (i, d) in enumerate(content['audio'])]
    audio_idx, _ = max(audio_quality, key=lambda _h: _h[1])
    video = content['video'][vid_idx]
    audio = content['audio'][audio_idx]
    base_url = base_url + content['base_url']
    video_tmp_file = 'video.mp4'
    audio_tmp_file = 'audio.mp4'
    self.fetch(video, video_tmp_file, base_url + video['base_url'])
    self.fetch(audio, audio_tmp_file, base_url + audio['base_url'])
    command = ["ffmpeg", "-i", audio_tmp_file, "-i", video_tmp_file, "-c", "copy", self.filename]
    try:
      result = subprocess.run(command, capture_output=True, text=True, check=True)
      print(result.stdout)
    except subprocess.CalledProcessError as e:
      print(f"Error: {e}")
      print(f"Output error: {e.stderr}")
    os.remove(video_tmp_file)
    os.remove(audio_tmp_file)


# @markdown ---
# @markdown \* SINGLE mode -> Download single video<br />
# @markdown \* PLAYLIST mode -> Download multiple videos from data.txt file<br />
# @markdown ***data.txt syntax (every line is new entry)***<br />
# @markdown *DATA_URL;OUTPUT_FILENAME;SAVE_TO_PC*
DOWNLOAD_MODE = "PLAYLIST" #@param ["SINGLE", "PLAYLIST"]
# @markdown ---
# @markdown Fill only if DOWNLOAD_MODE is SINGLE
DATA_URL = "[master|playlist].json file url"  # @param {type:"string"}
OUTPUT_FILENAME = "filename" # @param {type:"string"}
SAVE_TO_PC = False #@param {type:"boolean"}
# @markdown ---

data = []
if DOWNLOAD_MODE == "SINGLE":
  data.append((DATA_URL, OUTPUT_FILENAME, SAVE_TO_PC))
else:
  if not os.path.exists('data.txt'):
    print('No data.txt provided. Exiting...')
    sys.exit(1)
  with open('data.txt', 'r') as file:
    for i, line in enumerate(file.readlines(), start=1):
      videoData = [arg.strip() for arg in line.split(';')]
      if len(videoData) != 3 or videoData[0] == '' or videoData[1] == '' or videoData[2].capitalize() not in ['True', 'False']:
        print('Incorrect syntax in line', i)
        continue
      videoData[2] = videoData[2].capitalize() == 'True'
      data.append(tuple(videoData))

for DATA_URL, OUTPUT_FILENAME, SAVE_TO_PC in data:
  print(f"Downloading {OUTPUT_FILENAME}...")
  downloader = VimeoDownloader(DATA_URL, OUTPUT_FILENAME)
  downloader.download()
  if SAVE_TO_PC:
    downloader.save()

# @markdown ####How to find the playlist.json?
# @markdown #####Right click on the video > Inspect > Click on the Network tab > Play the video > Right click on "playlist.json?omit=..." > Copy the Request URL, something like this: vod-adaptive-ak.vimeocdn.com/exp=...


#@markdown ---
#@markdown #####Inspired by
#@markdown ##### https://gist.github.com/alexeygrigorev/a1bc540925054b71e1a7268e50ad55cd?permalink_comment_id=5097364#gistcomment-5097364